In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, utils
from torch.autograd import Variable
import os
from scipy.spatial.distance import cosine
import random

%matplotlib inline

# Util

In [30]:
def train_(im, target, model, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    output = model(im)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    return loss.item()

def test_(data, model, flatten=False):
    correct = 0
    total = 0
    model.eval()
    for im, target in data:
        if flatten:
            out = model(im.view(im.size(0), -1))
        else:
            out = model(im)
        pred = torch.max(out, dim=1)[1]
        correct += torch.eq(pred, target.view_as(pred)).sum().item()
        total += im.size(0)
    return 100.*correct/total

def train(train_loader, test_loader, model, optimizer, criterion, scheduler, flatten = False, epoch=10):
    for n in range(epoch):
        for i, (im, target) in enumerate(train_loader):
            if flatten:
#                 im_flatten = im.view(im.size(0), -1)
                loss = train_(im.view(im.size(0), -1), target, model, optimizer, criterion)
            else:
                loss = train_(im, target, model, optimizer, criterion)
            if i > 0 and i%500 == 0:
                val_acc = test_(test_loader, model, flatten=flatten)
#                 acc.append(val_acc)
                print('Epoch: [{}/{}], Step: [{}/{}], Training Loss: {:.2f},\
                        Validation Acc: {}'.format(
                                n+1, epoch, i+1, len(train_loader), loss, val_acc))
            
            

# Load Data

In [3]:
print('Loading MNIST')
train_loader = torch.utils.data.DataLoader(datasets.MNIST('data',
                                                            train=True,
                                                            download=True,
                                                            transform=transforms.Compose([
                                                            transforms.ToTensor(),
                                                            transforms.Normalize((0.1307,), (0.3081,))])
                                                        ),batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.MNIST('data',
                                                            train= False,
                                                            download=True,
                                                            transform=transforms.Compose([
                                                            transforms.ToTensor(),
                                                            transforms.Normalize((0.1307,), (0.3081,))])
                                                        ),batch_size=32, shuffle=True)
print('MNIST has been loaded.')


Loading MNIST
MNIST has been loaded.


# ConvNet

In [32]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # 10 channels in first convolution layer
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # 20 channels in second conv. layer
        self.fc1 = nn.Linear(320, 50) # 50 hidden units in first fully-connected layer
        self.fc2 = nn.Linear(50, 10) # 10 output units

    def forward(self, x):

        # first convolutional layer
        h_conv1 = self.conv1(x)
        h_conv1 = F.relu(h_conv1)
        h_conv1_pool = F.max_pool2d(h_conv1, 2)

        # second convolutional layer
        h_conv2 = self.conv2(h_conv1_pool)
        h_conv2 = F.relu(h_conv2)
        h_conv2_pool = F.max_pool2d(h_conv2, 2)

        # fully-connected layer
        h_fc1 = h_conv2_pool.view(-1, 320)
        h_fc1 = self.fc1(h_fc1)
        h_fc1 = F.relu(h_fc1)
        
        # classifier output
        output = self.fc2(h_fc1)
        output = F.log_softmax(output,dim=1)
        return output
#         return output, h_fc1, h_conv2, h_conv1



In [33]:
learning_rate = 0.01
ConvNet_model = ConvNet()
optimizer = torch.optim.SGD(ConvNet_model.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 0.95 ** epoch)

train(train_loader, test_loader, ConvNet_model, optimizer, criterion, scheduler, epoch=2)

Epoch: [1/2], Step: [501/1875], Training Loss: 0.23,                        Validation Acc: 85.77
Epoch: [1/2], Step: [1001/1875], Training Loss: 0.60,                        Validation Acc: 93.55
Epoch: [1/2], Step: [1501/1875], Training Loss: 0.32,                        Validation Acc: 95.09
Epoch: [2/2], Step: [501/1875], Training Loss: 0.06,                        Validation Acc: 96.67
Epoch: [2/2], Step: [1001/1875], Training Loss: 0.03,                        Validation Acc: 96.87
Epoch: [2/2], Step: [1501/1875], Training Loss: 0.01,                        Validation Acc: 97.31


In [39]:
ConvNet_total_params = sum(p.numel() for p in ConvNet_model.parameters())
print('Number of parameters in ConvNet model: {}'.format(ConvNet_total_params))

torch.save(ConvNet_model, 'model/ConvNet.pt')

Number of parameters in ConvNet model: 21840


# MLP

In [25]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(784, 128)
#         self.fc2 = nn.Linear(512, 216)
        self.fc3 = nn.Linear(128, 50)
        self.fc4 = nn.Linear(50, 10)

    def forward(self, x):
        h_fc1 = self.fc1(x)
        h_fc1 = F.relu(h_fc1)
        
#         h_fc2 = self.fc2(h_fc1)
#         h_fc2 = F.relu(h_fc2)
        
        h_fc3 = self.fc3(h_fc1)
        h_fc3 = F.relu(h_fc3)
        
        # classifier output
        output = self.fc4(h_fc3)
        output = F.log_softmax(output,dim=1)
        return output
#         return output, h_fc1, h_conv2, h_conv1


In [34]:
learning_rate = 0.01
MLP_model = MLP()
optimizer = torch.optim.SGD(MLP_model.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 0.95 ** epoch)

train(train_loader, test_loader, MLP_model, optimizer, criterion, scheduler)

RuntimeError: size mismatch, m1: [896 x 28], m2: [784 x 128] at /Users/distiller/project/conda/conda-bld/pytorch_1579022036889/work/aten/src/TH/generic/THTensorMath.cpp:136

In [27]:
MLP_total_params = sum(p.numel() for p in MLP_model.parameters())
print('Number of parameters in MLP model: {}'.format(MLP_total_params))



Number of parameters in MLP model: 107440


# PCA